In [2]:
import pandas as pd
df = pd.read_csv("../data/fewshot.multiturn.csv")
df.head()

,Conversations,Category
0,User: Show me sales and movement for Upper Res...,Sales
1,User: Can you give me the sales figures for Be...,Sales
2,User: I need to see the movement of Electronic...,Sales
3,User: How much revenue was generated from Home...,Sales
4,User: I want to see the sales numbers for Outd...,Sales


In [6]:
print(df.loc[0].Conversations)

User: Show me sales and movement for Upper Respiratory for Q2.
 AI: I need some more information to get the sales and movement for Upper Respiratory for Q2. What is the location you would like to know?
 Thought: User needs to provide location for the Movement API
 Observation: I need to ask the user for the location
 User: The location is Zone 1000
 AI: Got it. I will get the sales and movement for Upper Respiratory for Q2 in Zone 1000.
 Thought: User provided the location for the Movement API
 Observation: I need to call the Movement API with the provided information


In [7]:
categories = df.Category.unique().tolist()
categories

['Sales',
 'Budget Forecast',
 'KVI',
 'Price Change',
 'Sales Comparison',
 'Promo',
 'KVI Price Change',
 'Effective Promotions',
 'Price Comparison']

In [60]:
def create_chunks(df, n=3):

    def create_context(category):
        convs = df[df.Category==category].Conversations.sample(n).tolist()
        header = f"""
        {category}
        -------------
        """
        examples = "\n\n".join(convs)
        footer = """
        You are the AI. Based on the examples given above, converse with the user.
        Ask user for missing information. Do not ask questions about information you already have.
        Add observations, thoughts and actions wherever necessary."""
        chunk = header + "\n" + examples + "\n" + footer
        print(len(chunk))
        return chunk

    chunks = []
    for category in df.Category.unique().tolist():
        chunks.append(create_context(category))
    return chunks

text_chunks = create_chunks(df)

2570
2006
1278
1510
2431
2757
2473
2420
1757


## Question Answering from Context

In [56]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator

In [57]:
import os
OPENAI_API_KEY = open("../credentials/openai.key.txt").read().replace("\n", "").replace(" ", "")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [61]:
embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_texts(text_chunks, embeddings, metadatas=[{"source": str(i)} for i in range(len(text_chunks))])


Using embedded DuckDB without persistence: data will be transient


In [62]:
query = "How are we tracking against the budget goals for Deli products for the first half of the year?"
output = docsearch.similarity_search(query, k=1)

In [64]:
output[0].page_content

'\n        Budget Forecast\n        -------------\n        \nUser: What are the financial goals for Meat and Poultry?\n AI: What is the location and time period you would like to get the budget and forecast for?\n User: The location is Zone 1000 and I need the budget and forecast for the upcoming year.\n Thought: User specified the location and time period.\n Action: Call Budget API with {"product-name" = "Meat and Poultry", "time-period" = "year", "location-name" = "Zone 1000"}\n Action: Call Forecast API with {"product-name" = "Meat and Poultry", "time-period" = "year", "location-name" = "Zone 1000"}\n Action: Call Display API with {"type" = ["table", "text"], data = [Budget API Response + Forecast API response, "We are projected to meet the sales and movement goals, but fall short on margin."]}"\n\nUser: Can you tell me the budget goals for Frozen Foods?\n AI: Sure, I can help you with that. What is the location you would like to know about?\n User: The location is Zone 1000.\n Thou

In [65]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
query = "How are we tracking against the budget goals for Deli products for the first half of the year?"
docs = docsearch.similarity_search(query, k=1)
chain.run(input_documents=docs, question=query)

' What is the location you would like to know about?'

In [69]:
def interact():
    query = input("User: ")
    docs = docsearch.similarity_search(query, k=1)
    response = chain.run(input_documents=docs, question=query)
    while query != "exit":
        print("AI:", response)
        # update docs
        docs[0].page_content += f"\nUser: {query}\nAI: {response}"
        query = input("User: ")
        response = chain.run(input_documents=docs, question=query)

## Budget Forecast

In [70]:
interact()

User: How are we tracking against the budget goals for Deli products for the first half of the year?


Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


 What is the location you would like to know about?
User: Zone 1000
 I need the budget goals for the location in Zone 1000 for the first half of the year.
User: Yes
 Sure, I can help you with that. What is the location you would like to know about?
User: Zone 1000
 I have the budget goals for Deli products in Zone 1000 for the first half of the year. Would you like me to display the results?
User: Yes
 Sure, I can help you with that. I will call the Budget API with {"product-name" = "Deli products", "time-period" = "first half of the year", "location-name" = "Zone 1000"} and display the results.
User: exit


In [74]:
interact()

User: What are the financial goals for Meat and Poultry for the upcoming year?
 I can help you with that. What is the location you would like to know about?
User: Zone 300
 Sure, I can help you with that. What time period would you like to know about?
User: Upcoming year
 I have all the information I need. Action: Call Budget API with {"product-name" = "Meat and Poultry", "time-period" = "year", "location-name" = "Zone 300"}. Action: Call Forecast API with {"product-name" = "Meat and Poultry", "time-period" = "year", "location-name" = "Zone 300"}. Action: Call Display API with {"type" = ["table", "text"], data = [Budget API Response + Forecast API response, "We are projected to meet the sales and movement goals, but fall short on margin."]}" Would you like me to display the results?
User: Yes
 Sure, I can do that.
User: exit


## KVI's

In [75]:
interact()

User: Give me the Primary KVIs for Baby Products in Zone 20.
 I understand you need a list of Primary KVIs for Baby Products in Zone 20. I will call the KVI API with the following parameters: {"product-name" = "Baby Products", "location-name" = "Zone 20", type = "Primary"} and then call the Display API with {"type" = "list", data = KVI Response}. Is this correct?
User: exit


## Price change

In [76]:
interact()

User: What are the products that have had a price change in the last week for Meat and Poultry in Zone 450?
 I need to call Cost API with the given parameters. Call Cost API with {"product-name" = "Meat and Poultry", "location" = "Zone 450", "week" = "last 1", "change" = "Y"}
User: Yup
 Is there any other information you would like to provide?
User: exit


## Promotion Percent

In [77]:
interact()

User: What percentage of our Produce sales in Zone 20 are from promotions for the next six months?
 I'm sorry, I don't have the information to answer that question. Could you please provide me with the product name and the top number of promotions you would like to see?
User: Grocery
 I need to call Promo Analysis API for the given item, filter for the top three most effective promotions, and display results to the user. AI: To get the most effective promotions for Grocery in the next six months, I need to know the top number of promotions you would like to see. How many would you like?
User: 2
 I need to call Promo Analysis API for the given item, filter for the top two most effective promotions, and display results to the user. AI: To get the most effective promotions for Grocery in the next six months, I need to know the top number of promotions you would like to see. How many would you like?
User: 2
 I need to call Promo Analysis API for the given item, filter for the top two most 

## Sales comparison

In [78]:
interact()

User: Give me a comparison of sales for Personal Care products in Zone 750 for the current quarter and the same quarter last year.
 For which time period would you like to compare the sales figures?
User: Last year
 You would like to compare the sales figures for Personal Care products in Zone 750 for the current quarter and the same quarter last year. Is that correct?
User: yes
 Okay, I will call the Movement API with {"product-group" = "Personal Care", "time-period" = "quarter", "metrics" = ["sales"], "location-name" = "Zone 750"} and use the numerical format to display the sales figures.
User: exit


## Promotions

In [79]:
interact()

User: Give me a list of upcoming items with a BOGO 25% off offer in the Toys category. Which of these items are in Zone 950?
 I need the week, but user did not specify it. Action: Call Follow-Up Question API with {"type" = "week"}. AI: Which week would you like to know about?
User: last week
 I understand you would like to know about the items with a BOGO 25% off offer in the Toys category in Zone 950 for last week. Is that correct?
User: Yes
 Action: Call Promo API with {"product-name" = "Toys", "location-name" = "Zone 950", "week" = "last 1", "type" = "BOGO 25% off", "page-no" = 1} Action Call Display API with {"type" = "list", "data" = Promo API Response}
User: exit


## Effective Promotions

In [82]:
interact()

User: Which promotions have shown the best results for the Cleaning Supplies category in the last 2 months?
 To get the most effective promotions for Cleaning Supplies in the last 2 months, I need to know the top number of promotions you would like to see. How many would you like?
User: 3
 Sure, I can help you with that. I need to know the top number of promotions you would like to see. How many would you like?
User: 3
 Sure, I can help you with that. I need to call Promo Analysis API for the given item, filter for the top three most effective promotions, and display results to the user. Action: Call Promo Analysis API with {"product-name" = "Cleaning Supplies", "top" = 3} Action: Call Display API with {"type" = "table", "data" = Promo Analysis API Response}
User: exit
